In [ ]:
# Cell 11: Image Classification Dataset
class ImageClassificationDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        image = Image.open(image_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)
        
        if "vehicle" in image_path:
            label = 0
        else:
            label = -1  
        
        return image, label


vehicle_image_paths = [os.path.join("dataset", "train", "images", img) for img in os.listdir(os.path.join("dataset", "train", "images"))]
classification_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
dataset = ImageClassificationDataset(vehicle_image_paths, transform=classification_transform)
image, label = dataset[0]
print("Image shape:", image.shape)
print("Label:", label)

In [ ]:
# Cell 12: Image Classification Dataset Creation
classification_dataset = ImageClassificationDataset(vehicle_image_paths, transform=classification_transform)
classification_dataloader = DataLoader(classification_dataset, batch_size=16, shuffle=True)

for images, labels in classification_dataloader:
    print("Batch shape:", images.shape)
    print("Labels:", labels)
    break

In [ ]:
# Cell 13: Image Classification Model
class_names = ["vehicle"]
num_classes = len(class_names)

classification_model = models.resnet18(weights='IMAGENET1K_V1')
num_features = classification_model.fc.in_features
classification_model.fc = nn.Linear(num_features, num_classes)

classification_model.to(device)

# Capture the model architecture summary in a string
model_summary = str(classification_model)

print("Model architecture:")
print(model_summary)

In [ ]:
# Cell 14: Image Classification Training
classification_criterion = nn.CrossEntropyLoss()
classification_optimizer = torch.optim.Adam(classification_model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    classification_model.train()
    
    for images, labels in classification_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = classification_model(images)
        loss = classification_criterion(outputs, labels)
        
        classification_optimizer.zero_grad()
        loss.backward()
        classification_optimizer.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Classification Loss: {loss.item():.4f}")

torch.save(classification_model.state_dict(), "classification_model.pth")

classification_model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in classification_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = classification_model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Classification Accuracy: {accuracy:.2f}%")